# Data Preprocessing

Хамгийн эхлээд, [Моззиллагийн коммон войс датасеттайгаа](https://commonvoice.mozilla.org/en) танилцахад, train.tsv, test.tsv-д тухайн текстэд харгалзах аудио mp3 file-ын path байв. Ерөнхийд нь нэг нэгээр санамсаргүй байдлаар түүж сонсоход зарим нь нилээд шуугиантай, уншигчийн дуу сул зэрэг, ихэнхдээ чанар муу жишээнүүд байна гэж дүгнэсэн. Датасет дотор мөн validated_sentences.tsv жишээнүүдийн цуглуулга байгааг сонгоход ерөнхий train.tsv-ээс хамаагүй боломжийн чанартай санагдсан.

Машин сургалтад "garbage-in-garbage-out" зарчим буюу датаны чанар маш чухал учраас би зөвхөн  validated_sentences.tsv-ээс авсан train, test датасет жишээнүүд дээр ажиллахаар шийдсэн. Үүнд нийтдээ train-д 2188, test-д 1934-н жишээ буюу 50/50 split байв. Тийм учраас би train, test хоёр датасетээ нэгтгэж байгаад буцаагаад 80/20 харьцаатайгаар хуваасны үр дүнд, train 3297, test 825 жишээтэй болгов.

Түүнчлэн, Speech LLM-ын Acoustic Model-оор Whisper-г сонгож авсан учраас бүх mp3 file-уудыг 16kHz wav file-руу хөрвүүлсэн.

In [18]:
import pandas as pd

validated = pd.read_csv(
    "validated_sentences.tsv",
    sep="\t",
    usecols=["sentence_id"]
)

validated_ids = set(validated["sentence_id"])


In [19]:
def filter_validated(split_name):
    df = pd.read_csv(f"{split_name}.tsv", sep="\t")
    df = df[df["sentence_id"].isin(validated_ids)]
    return df[["path", "sentence", "sentence_id"]]

train_df = filter_validated("train")
test_df  = filter_validated("test")


In [20]:
print("train shape: ", train_df.shape)
print("test shape: ", test_df.shape)

train shape:  (2188, 3)
test shape:  (1934, 3)


In [21]:
train_df.head(1)

,path,sentence,sentence_id
0,common_voice_mn_21799896.mp3,хэмээн өөрийгөө тойрч зогссон шавь нартаа тунь...,d1a377990e5cbfb2dabba119451025a2bb1ed26035be50...


In [22]:
df = pd.concat([train_df,test_df])
df.shape

(4122, 3)

In [23]:
import os
import subprocess
from tqdm import tqdm

In [24]:
CLIPS_DIR = "./clips"
WAV_DIR   = "./wav_16k"
os.makedirs(WAV_DIR, exist_ok=True)

def convert_mp3_to_wav(mp3_path):
    wav_path = os.path.join(
        WAV_DIR,
        mp3_path.replace(".mp3", ".wav")
    )

    if os.path.exists(wav_path):
        return wav_path

    os.makedirs(os.path.dirname(wav_path), exist_ok=True)

    subprocess.run([
        "ffmpeg",
        "-y",
        "-i", os.path.join(CLIPS_DIR, mp3_path),
        "-ac", "1",
        "-ar", "16000",
        wav_path
    ], check=True)

    return wav_path

In [25]:
for p in tqdm(df["path"]):
        convert_mp3_to_wav(p)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4122/4122 [00:00<00:00, 226998.95it/s]


In [26]:
df["path"] = df["path"].str.replace(".mp3", ".wav")
df.to_csv("mn_16kHz_wav_validated_sentences.tsv", sep="\t", index=False)

In [27]:
df = pd.read_csv("mn_16kHz_wav_validated_sentences.tsv", delimiter='\t')
print(f"Total samples: {len(df)}")

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df, 
    test_size=0.2, 
    random_state=42,
    shuffle=True
)

train_df.to_csv("train.tsv", sep='\t', index=False)
test_df.to_csv("test.tsv", sep='\t', index=False)

Total samples: 4122


In [28]:
train_df.shape, test_df.shape

((3297, 3), (825, 3))